In [ ]:
# Import and leverage the imported file for analysis
import pandas as pd
import openpyxl
import os
import tkinter as tk
from tkinter import Tk
from tkinter import filedialog
import folium
import numpy as np
from h3 import h3

Tk().withdraw()
file_path = filedialog.askopenfilename()

In [ ]:
# Extract the base name of the file
file_name = os.path.basename(file_path)

# Remove the extension from the file name
file_name_without_extension = os.path.splitext(file_name)[0]

In [ ]:
# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

print(file_name)
print(df.columns)

In [ ]:
# Load data from Excel file into DataFrame
data = pd.read_excel(file_path)

In [ ]:
# Cleaning and preprocessing
df = df.dropna(subset=['latitude', 'longitude'])
numerical_cols = df.select_dtypes(include=[np.number]).columns
df[numerical_cols] = df[numerical_cols].applymap(lambda x: 0 if (pd.isnull(x) or np.isinf(x)) else x)

for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype(str)

date_cols = ['image_to_date', 'effective_date', 'Date of Loss']
for col in date_cols:
    if col in df.columns:
        df[col] = df[col].astype(str)

columns_to_drop = [
    'address2', 'Claim No', 'CAT Number', 'Policy Form', 'Replacement Cost Method',
    'Claim Estimate Cost (Full Replacement Cost)', 'Indemnity Paid', 'Indemnity Reserve',
    'Total Loss Adjustment Expense Paid', 'Defense Cost Containment Paid', 'Defense Cost Containment Reserve',
    'Public Adjuster', 'Attorney', 'Claim Status', 'Close Without Payment'
]
df = df.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
# Convert lat-lon to H3 indexes
resolution = 7
df['h3_index'] = df.apply(lambda row: h3.geo_to_h3(row['latitude'], row['longitude'], resolution), axis=1)

# Aggregate data by H3 index
aggregated_data = df.groupby('h3_index').agg({
    'zfire_AAL': 'sum',
    'TIV (Cov A+B+C)': 'sum',
    'external_id': 'count'
}).rename(columns={'external_id': 'property_count'}).reset_index()

In [ ]:
# Normalize the AAL by the mean
mean_aal = aggregated_data['zfire_AAL'].mean()
aggregated_data['normalized_AAL'] = aggregated_data['zfire_AAL'] / mean_aal

# Round the aggregate AAL to two decimal places
aggregated_data['zfire_AAL'] = aggregated_data['zfire_AAL'].round(2)

# Convert H3 back to lat-lon for visualization
aggregated_data['latitude'] = aggregated_data['h3_index'].apply(lambda x: h3.h3_to_geo(x)[0])
aggregated_data['longitude'] = aggregated_data['h3_index'].apply(lambda x: h3.h3_to_geo(x)[1])

In [ ]:
# Create the map base
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=10, tiles='CartoDB Positron')

# Create a feature group for properties
property_layer = folium.FeatureGroup(name="Property Addresses")

# Add circle markers with a fixed radius (in meters) for property addresses
for idx, row in df.iterrows():
    folium.Circle(
        location=[row['latitude'], row['longitude']],
        radius=5,  # Fixed radius
        tooltip=f"Level 1: {row['level_1']}<br>Level 2: {row['level_2']}<br>External ID: {row['external_id']}",
        color="black",
        fill=True,
        fill_color="black"
    ).add_to(property_layer)

# Create a feature group for hexagons
h3_layer = folium.FeatureGroup(name="H3 Hexagons")

# Use a logarithmic colormap to emphasize differences
from branca.colormap import linear

from branca.colormap import LinearColormap

# Create a custom linear colormap transitioning from green to red
colormap = LinearColormap(colors=['green', 'yellow', 'red'], index=[0, 0.5, aggregated_data['normalized_AAL'].max()], vmin=0, vmax=aggregated_data['normalized_AAL'].max()).to_step(10)
colormap.caption = "Normalized AAL"
m.add_child(colormap)

# Add hexagons with tooltips and popups to the feature group
for idx, row in aggregated_data.iterrows():
    hex_coords = h3.h3_to_geo_boundary(row['h3_index'])
    color = colormap(row['normalized_AAL'])  # Get the color for the cell based on the colormap

    popup_content = f"""
    <strong>H3 Index:</strong> {row['h3_index']}<br>
    <strong>Total Properties:</strong> {row['property_count']}<br>
    <strong>Aggregate AAL:</strong> ${row['zfire_AAL']:,}<br>
    <strong>Aggregate TIV:</strong> ${row['TIV (Cov A+B+C)']:,}<br>
    <strong>Normalized Aggregate AAL:</strong> {row['normalized_AAL']:.2f}
    """
    
    folium.Polygon(
        locations=hex_coords,
        fill=True,
        color="gray",  # or any other color for the faint border
        fill_color=color,
        fill_opacity=0.6,
        popup=folium.Popup(popup_content, max_width=300)
    ).add_to(h3_layer)

property_layer.add_to(m)
h3_layer.add_to(m)

# Add the layer control to the map and save
folium.LayerControl().add_to(m)
m.save("folium_map.html")
m

In [ ]:
# Export the DataFrames to an Excel file
with pd.ExcelWriter("combined_data.xlsx") as writer:
    aggregated_data.to_excel(writer, sheet_name="Aggregated H3 Data", index=False)
    df.to_excel(writer, sheet_name="Properties with H3", index=False)
